# Adding user-defined influence, reaction model and GUI: The segregation model

The segregation model has been proposed by [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling) in 1971 in his famous paper [Dynamic Models of Segregation](https://www.stat.berkeley.edu/~aldous/157/Papers/Schelling_Seg_Models.pdf). The goal of this model is to show that segregation can occur even if it is not wanted by the agents.

In our implementation of this model, turtles are located in the grid and at each step, compute an happiness index based on the similarity of other agents in their neighborhood. If this index is below a value, called here similarity rate, the turtle wants to move to an other location.

## Grab dependencies

In [2]:
#!ruby
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ-artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

java_import 'java.util.LinkedList'
java_import 'java.awt.geom.Point2D'
java_import 'java.util.ArrayList'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.levels.ExtendedLevel'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.libs.timemodel.PeriodicTimeModel'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.AgentCategory'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.LevelIdentifier'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.SimulationTimeStamp'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.IGlobalState'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.ILocalStateOfAgent'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.IPerceivedData'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.dynamicstate.ConsistentPublicLocalDynamicState'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.influences.IInfluence'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.influences.InfluencesMap'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.influences.RegularInfluence'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.levels.ILevel'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.initializations.AbstractLogoSimulationModel'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.LogoSimulationParameters'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.Parameter'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtlePLSInLogo'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.environment.LogoEnvPLS'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.levels.LogoDefaultReactionModel'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.model.ConeBasedPerceptionModel'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.tools.random.PRNG'

Resolving dependency: fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact com.sparkjava#spark-core;2.7.2!spark-core.jar(bundle)
Preparing to download artifact org.slf4j#slf4j-simple;1.7.13!slf4j-simple.jar
Preparing to download artifact org.apache.commons#commons-math3;3.6.1!commons-math3.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel-commonLibs;1.0-SNAPSHOT!similar-microKernel-commonLibs.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel;1.0-SNAPSHOT!similar-extendedKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel-extendedLibs;1

Java::FrLgi2aSimilar2logoLibToolsRandom::PRNG


## Model parameters

We define the following parameters and their default values.

In [3]:
class SegregationSimulationParameters < LogoSimulationParameters
  
  attr_accessor :similarityRate, :vacancyRate, :perceptionDistance
  def initialize
    
    @similarityRate = 3.0/8
  
    @vacancyRate = 0.05
  
    @perceptionDistance = Math::sqrt(2)
  end
  
end

No Outputs

## Model-specific influence

We define an influence called `Move` that is emitted by an agent who wants to move to another location. It is defined by a  unique identifier, here "move", and the state of the turtle that wants to move.

In [4]:
class Move < RegularInfluence
  
  attr_accessor :target
  def initialize( s, ns, target)
    super("move", LogoSimulationLevelList::LOGO, s, ns)
    @target = target
  end
  
end

No Outputs

## Decision model

The decision model computes a happiness index based on the rate of turtles of different categories in its neighborhood. If the index is below the parameter `similarityRate`, the turtle emits a `Move` influence.

In [5]:
class SegregationAgentDecisionModel < AbstractAgtDecisionModel
  
  def initialize(parameters)
    super(LogoSimulationLevelList::LOGO)
    @parameters = parameters
  end
  
  def decide(
    timeLowerBound,
    timeUpperBound,
    globalState,
    publicLocalState,
    privateLocalState,
    perceivedData,
    producedInfluences
  )
    sr = 0.0
    perceivedData.turtles.each do |perceivedTurtle|
      if perceivedTurtle.content.categoryOfAgent.isA(publicLocalState.categoryOfAgent)
        sr+=1.0
      end
    end
    if !perceivedData.turtles.empty
      sr/= perceivedData.turtles.size
    end
    if sr < @parameters.similarityRate
      producedInfluences.add(Move.new(timeLowerBound, timeUpperBound, publicLocalState))
    end
  end
end

No Outputs

## Reaction model

The reaction model handles the `Move` influences emitted by unhappy turtles. First, it identifies vacant places and moves the turtles that have emitted a `Move` influence. Note that if there is not enough vacant places, not all turtle wishes can be fulfilled.

In [6]:
class SegregationReactionModel < LogoDefaultReactionModel
  def makeRegularReaction(
      transitoryTimeMin,
      transitoryTimeMax,
      consistentState,
      regularInfluencesOftransitoryStateDynamics,
      remainingInfluences
    )
    if regularInfluencesOftransitoryStateDynamics.size > 2
      specificInfluences = ArrayList.new
      vacantPlaces = ArrayList.new
      specificInfluences.addAll(regularInfluencesOftransitoryStateDynamics)
      PRNG::get.shuffle(specificInfluences)
      for x in 0..consistentState.publicLocalStateOfEnvironment.width()-1
        for y in 0..consistentState.publicLocalStateOfEnvironment.height()-1
          if consistentState.publicLocalStateOfEnvironment.getTurtlesAt(x, y).empty?
            vacantPlaces.add(Point2D::Double.new(x,y))
          end
        end
      end
      PRNG::get.shuffle(vacantPlaces)
      i = 0
      specificInfluences.each do |influence|
        if influence.getCategory() == "move"
          consistentState.publicLocalStateOfEnvironment.turtlesInPatches[influence.target.location.x.floor][influence.target.location.y.floor].clear()
          consistentState.publicLocalStateOfEnvironment.turtlesInPatches[vacantPlaces[i].x.floor][vacantPlaces[i].y.floor].add(influence.target)
          influence.target.setLocation(vacantPlaces[i])
          i+=1      
        end
        if i >= vacantPlaces.size
          break
        end
      end
    end
  end 
end

No Outputs

## Simulation model

The simulation model generates the Logo level using the user-defined reaction model and a simple periodic time model. It also generates turtles of 2 different types (a and b) randomly in the grid with respect to the vacancy rate parameter.

In [7]:
class SegregationSimulationModel < AbstractLogoSimulationModel
  def generateLevels(p)
    logo = ExtendedLevel.new(
      p.initialTime,
      LogoSimulationLevelList::LOGO,
      PeriodicTimeModel.new(1,0, p.initialTime),
      SegregationReactionModel.new
    )
    levelList = LinkedList.new
    levelList.add(logo)
    return levelList
  end
  def generateAgents(p, levels)
     result =  AgentInitializationData.new
     t = ""
    for x in 0...p.gridWidth-1
      for y in 0..p.gridHeight-1
        if PRNG::get.randomDouble >= p.vacancyRate
          if PRNG::get.randomBoolean
            t = "a"
          else
            t = "b"
          end
          turtle = TurtleFactory.generate(
            ConeBasedPerceptionModel.new(p.perceptionDistance, 2*Math::PI, true, false, false),
            SegregationAgentDecisionModel.new(p),
            AgentCategory.new(t, TurtleAgentCategory::CATEGORY),
            0,
            0,
            0,
            x,
            y
          )
          result.agents.add( turtle )
        end  
      end
    end
    return result
  end
end

No Outputs

## Launch the HTML runner

The GUI is defined in a variable called `segregationgui`. Finally, we launch the web server with the above described GUI.

In [10]:
segregationgui = %{
    <canvas id='grid_canvas' class='center-block' width='400' height='400'></canvas>
    <script type='text/javascript'>
        drawCanvas = function (data) {
            var json = JSON.parse(data),
                canvas = document.getElementById('grid_canvas'),
                context = canvas.getContext('2d');
            context.clearRect(0, 0, canvas.width, canvas.height);
            for (var i = 0; i < json.agents.length; i++) {
                var centerX = json.agents[i].x * canvas.width;
                var centerY = json.agents[i].y * canvas.height;
                var radius = 2;
                if (json.agents[i].t == 'a') {
                    context.fillStyle = 'red';
                } else {
                    context.fillStyle = 'blue';
                }
                context.beginPath();
                context.arc(centerX, centerY, radius, 0, 2 * Math.PI, false);
                context.fill();
            }
        }
</script>}
runner = Similar2LogoHtmlRunner.new
runner.config.setCustomHtmlBodyFromString(segregationgui)
runner.config.setExportAgents(true)
runner.initializeRunner(SegregationSimulationModel.new(SegregationSimulationParameters.new))
runner.showView

No Outputs